In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

In [2]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [4]:
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import albumentations as A

class SatelliteDataset:
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.infer:
            if self.transform:
                image = self.transform(image)
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

def preprocess(image, mask):
    # Preprocessing steps for image and mask (if needed)
    image = image / 255.0  # Normalize image
    mask = mask / 255.0  # Normalize mask
    return image, mask

def load_and_preprocess_img(img_path, mask_rle):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
    image, mask = preprocess(image, mask)
    return image, mask

transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
    ]
)

def apply_transform(image, mask):
    augmented = transform(image=image.numpy(), mask=mask.numpy())
    return augmented['image'], augmented['mask']

dataset = SatelliteDataset(csv_file='/content/gdrive/MyDrive/train.csv', transform=apply_transform)

dataset = tf.data.Dataset.from_generator(
    lambda: dataset,
    output_signature=(
        tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, None), dtype=tf.float32)
    )
)

dataloader = dataset.map(load_and_preprocess_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataloader = dataset.batch(16)
dataloader = dataset.shuffle(buffer_size=1000)
dataloader = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


ModuleNotFoundError: No module named 'albumentations'

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

# Double Convolution Block 정의
def double_conv(in_channels, out_channels):
    return tf.keras.Sequential([
        layers.Conv2D(out_channels, 3, padding='same'),
        layers.ReLU(),
        layers.Conv2D(out_channels, 3, padding='same'),
        layers.ReLU()
    ])

# 간단한 U-Net 모델 정의
class UNet(tf.keras.Model):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = layers.MaxPooling2D(2)
        self.upsample = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = layers.Conv2D(1, 1)

    def call(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)

        x = self.dconv_down4(x)

        x = self.upsample(x)
        x = tf.concat([x, conv3], axis=-1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = tf.concat([x, conv2], axis=-1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = tf.concat([x, conv1], axis=-1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

In [ ]:
# model 초기화
model = UNet()

# loss function 정의
loss_object = tf.keras.losses.BinaryCrossentropy()

# optimizer 정의
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(images, masks):
    with tf.GradientTape() as tape:
        outputs = model(images, training=True)
        loss = loss_object(masks, outputs)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

# training loop
for epoch in range(10):  # 10 에폭 동안 학습합니다.
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = tf.cast(images, tf.float32)
        masks = tf.cast(masks, tf.float32)

        loss = train_step(images, masks)
        epoch_loss += loss

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')